# KOSIS 주택 준공 실적 데이터 수집 및 분석

이 노트북은 PublicDataReader 라이브러리를 사용하여 국토교통부의 주택유형별 주택건설 준공실적 데이터를 수집하고 분석합니다.

**데이터 특성:**
- KOSIS 준공 데이터는 '월계'로 표시되어 있으며, 월별 실적을 의미합니다 (누계 아님)
- 착공실적과 마찬가지로 월별 실제 실적 데이터이므로 차분 계산이 필요하지 않습니다

**데이터 범위:**
- 기간: 2010년 8월 ~ 2025년 12월 (최신월까지)
- 지역: 전국 시도별 (17개 시도)
- 주택유형: 아파트, 단독, 다가구, 연립, 다세대

## 1. 라이브러리 Import

In [10]:
from PublicDataReader import Kosis
import pandas as pd
from datetime import datetime

# 그래프 라이브러리 (Plotly 사용)
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Plotly 기본 설정
import plotly.io as pio
pio.templates.default = "plotly_white"

print("✓ 라이브러리 Import 완료")
print("✓ Plotly 인터랙티브 그래프 사용")

✓ 라이브러리 Import 완료
✓ Plotly 인터랙티브 그래프 사용


## 2. 데이터 수집 함수 정의

In [ ]:
def collect_construction_data():
    """
    시도별 주택 준공 실적 데이터를 수집하는 함수
    KOSIS API의 40,000셀 제한을 고려하여 지역을 그룹으로 나눠서 수집

    Returns:
        pd.DataFrame: 수집된 준공 실적 데이터프레임
    """

    # KOSIS API 인스턴스 생성
    api_key = "M2ZlYmQxMzFlNmMwOGUwODVhMmZjOTIxNmE3ZjAyYTI="
    api = Kosis(api_key)

    print("KOSIS API 인스턴스 생성 완료")

    # 데이터 수집 파라미터 설정
    org_id = "116"  # 국토교통부
    tbl_id = "DT_MLTM_5373"  # 주택유형별 주택건설 준공실적

    # 지역 코드 리스트
    region_groups = [
        {"name": "총계", "codes": ["13102766973A.0001"]},
        {"name": "수도권소계", "codes": ["13102766973A.0002"]},
        {"name": "서울", "codes": ["13102766973A.0003"]},
        {"name": "인천", "codes": ["13102766973A.0004"]},
        {"name": "경기", "codes": ["13102766973A.0005"]},
        {"name": "지방소계", "codes": ["13102766973A.0006"]},
        {"name": "기타광역시", "codes": ["13102766973A.0007"]},
        {"name": "부산", "codes": ["13102766973A.0008"]},
        {"name": "대구", "codes": ["13102766973A.0009"]},
        {"name": "광주", "codes": ["13102766973A.0010"]},
        {"name": "대전", "codes": ["13102766973A.0011"]},
        {"name": "울산", "codes": ["13102766973A.0012"]},
        {"name": "기타지방", "codes": ["13102766973A.0013"]},
        {"name": "세종", "codes": ["13102766973A.0014"]},
        {"name": "강원", "codes": ["13102766973A.0015"]},
        {"name": "충북", "codes": ["13102766973A.0016"]},
        {"name": "충남", "codes": ["13102766973A.0017"]},
        {"name": "전북", "codes": ["13102766973A.0018"]},
        {"name": "전남", "codes": ["13102766973A.0019"]},
        {"name": "경북", "codes": ["13102766973A.0020"]},
        {"name": "경남", "codes": ["13102766973A.0021"]},
        {"name": "제주", "codes": ["13102766973A.0022"]},
    ]

    # 기간을 두 구간으로 나누기 (API 제한 대응)
    date_periods = [
        {"start": "201008", "end": "201812"},
        {"start": "201901", "end": "202512"},
    ]

    print(f"데이터 수집 시작: 201008 ~ 202512")
    print(f"총 {len(region_groups)}개 지역 x {len(date_periods)}개 기간으로 나누어 수집합니다.\n")

    all_dataframes = []
    total_requests = len(region_groups) * len(date_periods)
    current_request = 0

    try:
        for region_idx, group in enumerate(region_groups, 1):
            group_name = group["name"]
            obj_l1 = "+".join(group["codes"])

            for period in date_periods:
                current_request += 1
                start_date = period["start"]
                end_date = period["end"]

                print(f"[{current_request}/{total_requests}] {group_name} ({start_date[:4]}-{end_date[:4]}) 수집 중...")

                df_group = api.get_data(
                    "통계자료",
                    orgId=org_id,
                    tblId=tbl_id,
                    objL1=obj_l1,
                    objL2="ALL",
                    objL3="ALL",
                    objL4="ALL",
                    itmId="ALL",
                    prdSe="M",
                    startPrdDe=start_date,
                    endPrdDe=end_date,
                )

                if df_group is not None and not df_group.empty:
                    df_group = df_group.astype({'수치값': int})
                    all_dataframes.append(df_group)
                    print(f"  ✓ 수집 완료: {len(df_group)}건\n")
                else:
                    print(f"  ✗ 데이터가 없습니다.\n")

        if all_dataframes:
            df = pd.concat(all_dataframes, ignore_index=True)
            print(f"데이터 수집 완료! 총 {len(df):,}건의 데이터가 수집되었습니다.")
            return df
        else:
            print("수집된 데이터가 없습니다.")
            return None

    except Exception as e:
        print(f"데이터 수집 중 오류 발생: {str(e)}")
        return None

## 3. 데이터 변환 함수 정의

In [ ]:
def create_final_pivot_table(df, filename=None):
    """
    최종 분석용 피벗 테이블을 생성하는 함수

    주의: KOSIS 준공 데이터는 '월계'로 표시되어 있으며, 이는 월별 실적을 의미합니다.
    (인허가의 '월별 누계'와 달리 누적 데이터가 아니므로 차분 계산 불필요)

    전체 기간 데이터를 대상으로:
    - 소계/기타/총계 제외
    - 시도명을 정식 명칭으로 변경
    - 불필요한 컬럼 제거
    - Long format으로 변환 (시점, 시도, 주택유형, 개수)

    Args:
        df (pd.DataFrame): 원본 데이터프레임
        filename (str, optional): 저장할 파일명
    """
    if df is None or df.empty:
        print("저장할 데이터가 없습니다.")
        return

    try:
        print("\n=== 최종 피벗 테이블 생성 중 ===")
        print(f"수집 기간: {df['수록시점'].min()} ~ {df['수록시점'].max()}")

        # 소계/기타/총계 제외 (모든 시점)
        df_filtered = df[~df['분류값명1'].str.contains('소계', na=False)].copy()
        df_filtered = df_filtered[~df_filtered['분류값명1'].str.contains('기타', na=False)]
        df_filtered = df_filtered[df_filtered['분류값명1'] != '총계']

        # 피벗 테이블 생성 (시점별, 시도별)
        pivot_df = df_filtered.pivot_table(
            index=['수록시점', '분류값명1'],
            columns='분류값명4',
            values='수치값',
            aggfunc='sum',
            fill_value=0
        ).reset_index()

        # 시점 포맷 변경 (202510 -> 2025.10)
        pivot_df['시점'] = pivot_df['수록시점'].apply(lambda x: f"{str(x)[:4]}.{str(x)[4:]}")
        pivot_df = pivot_df.drop(columns=['수록시점'])

        # 시도명을 정식 명칭으로 변경
        sido_mapping = {
            '서울': '서울특별시',
            '부산': '부산광역시',
            '대구': '대구광역시',
            '인천': '인천광역시',
            '광주': '광주광역시',
            '대전': '대전광역시',
            '울산': '울산광역시',
            '세종': '세종특별자치시',
            '경기': '경기도',
            '충북': '충청북도',
            '충남': '충청남도',
            '전남': '전라남도',
            '경북': '경상북도',
            '경남': '경상남도',
            '제주': '제주특별자치도',
            '강원': '강원특별자치도',
            '전북': '전북특별자치도'
        }
        pivot_df['분류값명1'] = pivot_df['분류값명1'].map(sido_mapping)

        # 컬럼명 변경
        pivot_df = pivot_df.rename(columns={
            '분류값명1': '시도',
            '가구수': '다가구'
        })

        # 불필요한 컬럼 제거
        columns_to_keep = ['시점', '시도', '아파트', '단독', '다가구', '연립', '다세대']
        available_columns = [col for col in columns_to_keep if col in pivot_df.columns]
        pivot_df = pivot_df[available_columns]

        # Long format으로 변환
        df_long = pivot_df.melt(
            id_vars=['시점', '시도'],
            value_vars=['아파트', '단독', '다가구', '연립', '다세대'],
            var_name='주택유형',
            value_name='개수'
        )

        # 파일명 생성
        if filename is None:
            filename = "준공실적_피벗_전체기간_최종.csv"

        # csv 폴더에 저장
        filepath = f"../csv/{filename}"

        # CSV로 저장
        df_long.to_csv(filepath, index=False, encoding='utf-8-sig')
        print(f"최종 피벗 테이블이 '{filepath}' 파일로 저장되었습니다.")
        print(f"- 총 {len(df_long)}개 행 ({len(df_long['시점'].unique())}개 시점 x {len(df_long['시도'].unique())}개 시도 x {len(df_long['주택유형'].unique())}개 주택유형)")

        return df_long

    except Exception as e:
        print(f"최종 피벗 테이블 생성 중 오류 발생: {str(e)}")
        return None

## 4. 데이터 수집 실행 (선택사항)

**주의:** 이 셀을 실행하면 KOSIS API로부터 데이터를 새로 수집합니다 (약 2-3분 소요).  
이미 수집된 CSV 파일이 있다면 다음 셀에서 파일을 직접 로드하세요.

In [ ]:
# 새로 데이터 수집
df_raw = collect_construction_data()

if df_raw is not None:
    # 원본 데이터 저장
    current_date = datetime.now().strftime("%Y%m%d")
    df_raw.to_csv(f"../csv/준공실적_시도별_{current_date}.csv", index=False, encoding='utf-8-sig')
    print(f"원본 데이터가 '../csv/준공실적_시도별_{current_date}.csv' 파일로 저장되었습니다.")

## 5. 기존 데이터 로드 (선택사항)

이미 수집된 CSV 파일을 로드합니다.

In [ ]:
# 기존 원본 데이터 로드
df_raw = pd.read_csv('../csv/준공실적_시도별_20251226.csv', encoding='utf-8-sig')
print(f"원본 데이터 로드 완료: {len(df_raw):,}건")
print(f"수집 기간: {df_raw['수록시점'].min()} ~ {df_raw['수록시점'].max()}")

## 6. 최종 피벗 테이블 생성

In [14]:
# 최종 피벗 테이블 생성
df_final = create_final_pivot_table(df_raw)

# 데이터 확인
print("\n=== 최종 데이터 미리보기 ===")
display(df_final.head(20))

print("\n=== 데이터 통계 ===")
print(f"총 행 수: {len(df_final):,}")
print(f"시점 범위: {df_final['시점'].min()} ~ {df_final['시점'].max()}")
print(f"시도 목록: {', '.join(sorted(df_final['시도'].unique()))}")
print(f"주택유형: {', '.join(df_final['주택유형'].unique())}")


=== 최종 피벗 테이블 생성 중 ===
수집 기간: 201101 ~ 202510
최종 피벗 테이블이 '착공실적_피벗_전체기간_최종.csv' 파일로 저장되었습니다.
- 총 15040개 행 (178개 시점 x 17개 시도 x 5개 주택유형)

=== 최종 데이터 미리보기 ===


,시점,시도,주택유형,개수
0,2011.01,강원특별자치도,아파트,639
1,2011.01,경기도,아파트,315
2,2011.01,경상남도,아파트,271
3,2011.01,경상북도,아파트,57
4,2011.01,광주광역시,아파트,0
5,2011.01,대구광역시,아파트,0
6,2011.01,대전광역시,아파트,518
7,2011.01,부산광역시,아파트,1309
8,2011.01,서울특별시,아파트,103
9,2011.01,울산광역시,아파트,268



=== 데이터 통계 ===
총 행 수: 15,040
시점 범위: 2011.01 ~ 2025.10
시도 목록: 강원특별자치도, 경기도, 경상남도, 경상북도, 광주광역시, 대구광역시, 대전광역시, 부산광역시, 서울특별시, 세종특별자치시, 울산광역시, 인천광역시, 전라남도, 전북특별자치도, 제주특별자치도, 충청남도, 충청북도
주택유형: 아파트, 단독, 다가구, 연립, 다세대


## 7. 데이터 탐색

In [ ]:
# 특정 시도의 데이터 확인
sido_name = '서울특별시'
df_seoul = df_final[df_final['시도'] == sido_name]

print(f"\n=== {sido_name} 데이터 ===")
display(df_seoul.head(20))

# 주택유형별 총합
print(f"\n=== {sido_name} 주택유형별 총 준공 실적 (2011-2025) ===")
housing_total = df_seoul.groupby('주택유형')['개수'].sum().sort_values(ascending=False)
display(housing_total)

## 8. 시각화

### 8.1. 전국 주택유형별 준공 실적 추이

In [ ]:
# 시점을 datetime으로 변환
df_final['시점_dt'] = pd.to_datetime(df_final['시점'], format='%Y.%m')

# 전국 주택유형별 월별 합계
df_monthly = df_final.groupby(['시점_dt', '주택유형'])['개수'].sum().reset_index()

# 아파트와 비아파트로 분리
df_apt = df_monthly[df_monthly['주택유형'] == '아파트']
df_non_apt = df_monthly[df_monthly['주택유형'] != '아파트']

# 서브플롯 생성 (1행 2열)
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('아파트 준공 실적', '비아파트 준공 실적 (단독, 다가구, 연립, 다세대)'),
    horizontal_spacing=0.1
)

# 1. 아파트 그래프
fig.add_trace(
    go.Scatter(
        x=df_apt['시점_dt'],
        y=df_apt['개수'],
        mode='lines+markers',
        name='아파트',
        line=dict(width=1, color='#E74C3C'),
        marker=dict(size=3),
        hovertemplate='%{x|%Y.%m}<br>아파트: %{y:,.0f}<extra></extra>'
    ),
    row=1, col=1
)

# 2. 비아파트 그래프
colors = {'단독': '#3498DB', '다가구': '#2ECC71', '연립': '#F39C12', '다세대': '#9B59B6'}
for housing_type in ['단독', '다가구', '연립', '다세대']:
    df_type = df_non_apt[df_non_apt['주택유형'] == housing_type]
    fig.add_trace(
        go.Scatter(
            x=df_type['시점_dt'],
            y=df_type['개수'],
            mode='lines+markers',
            name=housing_type,
            line=dict(width=1, color=colors.get(housing_type)),
            marker=dict(size=3),
            hovertemplate=f'%{{x|%Y.%m}}<br>{housing_type}: %{{y:,.0f}}<extra></extra>'
        ),
        row=1, col=2
    )

# X축 설정 (레이블 회전)
fig.update_xaxes(
    title_text='시점', 
    tickangle=-90,
    dtick="M12",
    row=1, col=1
)
fig.update_xaxes(
    title_text='시점', 
    tickangle=-90,
    dtick="M12",
    row=1, col=2
)

# Y축 설정 (천단위 구분 기호)
fig.update_yaxes(
    title_text='준공 실적 (개수)', 
    tickformat=",",
    row=1, col=1
)
fig.update_yaxes(
    title_text='준공 실적 (개수)', 
    tickformat=",",
    row=1, col=2
)

# 전체 레이아웃 설정
fig.update_layout(
    title_text='전국 주택유형별 준공 실적 추이 (2011-2025)',
    title_font_size=16,
    height=500,
    hovermode='x unified',
    font=dict(size=12),
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.35,
        xanchor="center",
        x=0.5
    )
)

fig.show()

### 8.2. 시도별 준공 실적 비교 (최근 1년)

In [ ]:
# 최근 1년 데이터 필터링
latest_date = df_final['시점_dt'].max()
one_year_ago = latest_date - pd.DateOffset(months=12)
df_recent = df_final[df_final['시점_dt'] > one_year_ago]

# 아파트와 비아파트로 분리하여 시도별 합계 계산
df_apt_recent = df_recent[df_recent['주택유형'] == '아파트'].groupby('시도')['개수'].sum().reset_index()
df_apt_recent.columns = ['시도', '아파트']

df_non_apt_recent = df_recent[df_recent['주택유형'] != '아파트'].groupby('시도')['개수'].sum().reset_index()
df_non_apt_recent.columns = ['시도', '비아파트']

# 두 데이터프레임 합치기
df_sido_total = df_apt_recent.merge(df_non_apt_recent, on='시도', how='outer').fillna(0)
df_sido_total['합계'] = df_sido_total['아파트'] + df_sido_total['비아파트']
df_sido_total = df_sido_total.sort_values('합계', ascending=True)  # 오름차순 정렬

# 누적 가로 막대 그래프
fig = go.Figure()

# 아파트 막대
fig.add_trace(go.Bar(
    x=df_sido_total['아파트'],
    y=df_sido_total['시도'],
    orientation='h',
    name='아파트',
    marker=dict(color='#E74C3C'),
    text=df_sido_total['아파트'].apply(lambda x: f'{x:,.0f}' if x > 0 else ''),
    textposition='inside',
    textfont=dict(color='white'),
    hovertemplate='아파트: %{x:,.0f}<extra></extra>'
))

# 비아파트 막대
fig.add_trace(go.Bar(
    x=df_sido_total['비아파트'],
    y=df_sido_total['시도'],
    orientation='h',
    name='비아파트',
    marker=dict(color='#3498DB'),
    text=df_sido_total['비아파트'].apply(lambda x: f'{x:,.0f}' if x > 0 else ''),
    textposition='inside',
    textfont=dict(color='white'),
    hovertemplate='비아파트: %{x:,.0f}<extra></extra>'
))

# X축 숫자 형식 설정
fig.update_xaxes(tickformat=",")

# 레이아웃 설정
fig.update_layout(
    barmode='stack',  # 누적 막대
    title=f'시도별 주택 준공 실적 (최근 1년: {one_year_ago.strftime("%Y.%m")} ~ {latest_date.strftime("%Y.%m")})',
    xaxis_title='준공 실적 (개수)',
    yaxis_title='시도',
    height=600,
    font=dict(size=12),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig.show()

### 8.3. 특정 시도의 주택유형별 추이

In [ ]:
# 사용자로부터 시도 입력 받기
print("시도 정식 명칭을 입력하세요. (예: 서울특별시, 세종특별자치시, 전북특별자치도)")
print(f"\n사용 가능한 시도:")
for sido in sorted(df_final['시도'].unique()):
    print(f"  - {sido}")

target_sido = input("\n시도 입력: ")

# 입력된 시도가 유효한지 확인
if target_sido not in df_final['시도'].unique():
    print(f"\n⚠️ '{target_sido}'는 데이터에 없습니다.")
    print(f"사용 가능한 시도: {', '.join(sorted(df_final['시도'].unique()))}")
else:
    df_sido = df_final[df_final['시도'] == target_sido].copy()
    
    # 아파트와 비아파트로 분리
    df_sido_apt = df_sido[df_sido['주택유형'] == '아파트']
    df_sido_non_apt = df_sido[df_sido['주택유형'] != '아파트']
    
    # 서브플롯 생성 (1행 2열)
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=(f'{target_sido} 아파트 준공 실적', f'{target_sido} 비아파트 준공 실적'),
        horizontal_spacing=0.1
    )
    
    # 1. 아파트 그래프
    fig.add_trace(
        go.Scatter(
            x=df_sido_apt['시점_dt'],
            y=df_sido_apt['개수'],
            mode='lines+markers',
            name='아파트',
            line=dict(width=1, color='#E74C3C'),
            marker=dict(size=3),
            hovertemplate='%{x|%Y.%m}<br>아파트: %{y:,.0f}<extra></extra>'
        ),
        row=1, col=1
    )
    
    # 2. 비아파트 그래프
    colors = {'단독': '#3498DB', '다가구': '#2ECC71', '연립': '#F39C12', '다세대': '#9B59B6'}
    for housing_type in ['단독', '다가구', '연립', '다세대']:
        df_type = df_sido_non_apt[df_sido_non_apt['주택유형'] == housing_type]
        fig.add_trace(
            go.Scatter(
                x=df_type['시점_dt'],
                y=df_type['개수'],
                mode='lines+markers',
                name=housing_type,
                line=dict(width=1, color=colors.get(housing_type)),
                marker=dict(size=3),
                hovertemplate=f'%{{x|%Y.%m}}<br>{housing_type}: %{{y:,.0f}}<extra></extra>'
            ),
            row=1, col=2
        )
    
    # X축 설정 (레이블 회전)
    fig.update_xaxes(
        title_text='시점', 
        tickangle=-90,
        dtick="M12",
        row=1, col=1
    )
    fig.update_xaxes(
        title_text='시점', 
        tickangle=-90,
        dtick="M12",
        row=1, col=2
    )
    
    # Y축 설정 (천단위 구분 기호)
    fig.update_yaxes(
        title_text='준공 실적 (개수)', 
        tickformat=",",
        row=1, col=1
    )
    fig.update_yaxes(
        title_text='준공 실적 (개수)', 
        tickformat=",",
        row=1, col=2
    )
    
    # 전체 레이아웃 설정
    fig.update_layout(
        title_text=f'{target_sido} 주택유형별 준공 실적 추이 (2011-2025)',
        title_font_size=16,
        height=500,
        hovermode='x unified',
        font=dict(size=12),
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.35,
            xanchor="center",
            x=0.5
        )
    )
    
    fig.show()
    print(f"\n✓ {target_sido} 그래프가 생성되었습니다.")

### 8.4. 주택유형별 시도 비교 (최근 1년)

In [ ]:
# 주택유형별로 서브플롯 생성
housing_types = sorted(df_final['주택유형'].unique())

# 2행 3열 서브플롯 생성
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=[f'{ht} 준공 실적 (상위 10개 시도)' for ht in housing_types],
    horizontal_spacing=0.12,
    vertical_spacing=0.15
)

for idx, housing_type in enumerate(housing_types):
    row = idx // 3 + 1
    col = idx % 3 + 1
    
    # 해당 주택유형의 최근 1년 데이터
    df_type = df_recent[df_recent['주택유형'] == housing_type]
    df_type_sido = df_type.groupby('시도')['개수'].sum().reset_index()
    df_type_sido = df_type_sido.nlargest(10, '개수').sort_values('개수', ascending=True)
    
    # 가로 막대 그래프 추가
    fig.add_trace(
        go.Bar(
            x=df_type_sido['개수'],
            y=df_type_sido['시도'],
            orientation='h',
            text=df_type_sido['개수'].apply(lambda x: f'{x:,.0f}'),
            textposition='inside',  # 막대 안쪽
            textfont=dict(color='white'),  # 흰색 텍스트
            marker=dict(color=df_type_sido['개수'], colorscale='Teal'),
            showlegend=False,
            hovertemplate='%{y}: %{x:,.0f}<extra></extra>'
        ),
        row=row, col=col
    )
    
    # 각 서브플롯의 X축 설정 (천단위 구분 기호)
    fig.update_xaxes(
        title_text='준공 실적 (개수)', 
        tickformat=",",  # 천단위 구분 기호
        row=row, col=col
    )

# 전체 레이아웃 설정
fig.update_layout(
    title_text='주택유형별 시도 준공 실적 비교 (최근 1년)',
    title_font_size=16,
    height=800,
    font=dict(size=10),
    showlegend=False
)

fig.show()

## 9. 추가 분석 영역

아래 셀에 추가 분석 및 시각화 코드를 작성하세요.

In [ ]:
# 추가 분석 코드를 여기에 작성하세요
